In [1]:
import pandas as pd

import re

import demoji

from nltk.tokenize import TreebankWordTokenizer

import string

from nltk.corpus import stopwords

from gensim.models import Word2Vec

In [2]:
data = pd.read_csv('IRAhandle_tweets_1.csv',usecols=['content','language'])

In [3]:
tweets_en_rs = data[(data['language']=='English') | (data['language']=='Russian') ].content

In [4]:
%%time
emojis_list         = []
hashtags_list       = []
cleaned_tweets_list = []
words_list          = []

for tweet in tweets_en_rs:
    
    # urls
    tweet = re.sub(r'(https?:\/\/)(\s)*(www\.)?(\s)*((\w|\s)+\.)*([\w\-\s]+\/)*([\w\-]+)((\?)?[\w\s]*=\s*[\w\%&]*)*', '', tweet, flags=re.MULTILINE)
 
# uncomment
#     # emojis
#     emojis_in_tweet =  demoji.findall(tweet)
#     emojis_list.append(emojis_in_tweet)
    
    # hashtags
    hashtags_in_tweet = re.findall('\#(\w*)', tweet)
    hashtags_list.append(hashtags_in_tweet)
    
    # punctuations
    tweet = tweet.translate(str.maketrans('', '', string.punctuation))
    
    # numbers
    tweet = re.sub(r'\d+', '', tweet)
    # to lowercase
    tweet = tweet.lower()
    
    tweet = tweet.translate(str.maketrans('', '', string.punctuation))
    
    #check for tweet == null before appending 
    if tweet:
        cleaned_tweets_list.append(tweet)
    
    # stop-words
    stop_words = stopwords.words('english')
    
    tweet_to_words = TreebankWordTokenizer().tokenize(tweet)
    words__in_tweet = [words for words in tweet_to_words if words not in stop_words]    
    

Wall time: 1min 26s


In [5]:
# hashtags_list
# len(cleaned_tweets_list)


In [6]:
#TF-IDF

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

In [8]:
# %%time
# vectorizer = TfidfVectorizer()
# X = vectorizer.fit_transform(cleaned_tweets_list)

In [9]:
%%time

#instantiate CountVectorizer()
cv=CountVectorizer()
 
# this steps generates word counts for the words in your docs
word_count_vector=cv.fit_transform(cleaned_tweets_list)
 

tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)
 

# print idf values
df_idf = pd.DataFrame(tfidf_transformer.idf_, index=cv.get_feature_names(),columns=["idf_weights"])
 
# sort ascending
df_idf.sort_values(by=['idf_weights'])

# count matrix
count_vector=cv.transform(cleaned_tweets_list)
 
# tf-idf scores
tf_idf_vector=tfidf_transformer.transform(count_vector)

feature_names = cv.get_feature_names()
 


Wall time: 7.34 s


In [26]:
            #### TF-IDF    
    

#get tfidf vector for first document
first_document_vector=tf_idf_vector[0]
second_document_vector=tf_idf_vector[1]

#print the scores
# df = pd.DataFrame(first_document_vector.T.todense(), index=feature_names, columns=["TF-IDF"])
# df.sort_values(by=["TF-IDF"],ascending=False)

# similarity(first_document_vector,second_document_vector)

array([[0.00529199]])

In [15]:
# similarity between 2 docs

def similarity(doc1,doc2):
    doc1 = first_document_vector.toarray()
    doc2 = second_document_vector.toarray()
    return np.dot(doc1,doc2.T)

In [ ]:
        #### word2vector

In [87]:
%%time
w2v = Word2Vec(words_list,min_count=1)

Wall time: 23.8 s


In [113]:
w2v.wv.most_similar(['donald','trump'])

[('pence', 0.908073365688324),
 ('ferociously', 0.8967700004577637),
 ('j', 0.89599609375),
 ('president', 0.8923373222351074),
 ('rick', 0.8906669616699219),
 ('message', 0.8905045986175537),
 ('presidentelect', 0.8782235383987427),
 ('trump…', 0.8765853643417358),
 ('iowa', 0.8765603303909302),
 ('trumpsfavoriteheadline', 0.8760970830917358)]